# Import library

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
from sklearn import impute
warnings.filterwarnings("ignore")

# Load dataset

In [4]:
df = pd.read_json("../data/demo.json")
df = df.drop(['start_date'], axis = 1)
df = df.drop_duplicates()
df = df.drop(['end_date','description'], axis = 1)
df.shape



(5, 16)

In [5]:
def convert_area(area):
    for i in range(len(area)):
        try:
            if re.search("m$", area[i]):
                area[i] = re.sub("m", "", area[i])
                area[i] = re.sub(",", ".", area[i])
                area[i] = float(area[i])
            else:
                area[i] = np.nan
        except:
            area[i] = np.nan
    return area

def convert_direction(direction):
    for i in range(len(direction)):
        if (direction[i] == '_'):
            direction[i] = np.nan
        else:
            continue
    return direction

def convert_law(law):
    for i in range(len(law)):
        if law[i] == "---":
            law[i] = np.nan
        else:
            continue
    return law
        
def convert_price(price):
    for i in range(len(price)):
        try:
            if re.search("tỷ $", price[i]):
                price[i] = price[i].split(' ')[1]
                price[i] = re.sub(",", ".", price[i])
                price[i] = float(price[i])
            else:
                price[i] = np.nan
        except:
            price[i] = np.nan
    return price

def convert_address(address):
    city = []
    district = []
    for i in range(len(address)):
        address[i] = re.split(', ', address[i])
        try:
            if re.search('Quận ', address[i][-2]):
                address[i][-2] = re.sub('Quận ', '', address[i][-2])
            else:
                address[i][-2] = re.sub('Huyện ', '', address[i][-2])
        except:
            address[i][-2] = np.nan
        district.append(address[i][-2])
        city.append(address[i][-1])
        
    return city, district

In [6]:
price = df['price'].tolist()
area = df['area'].tolist()
width = df['width'].tolist()
height = df['height'].tolist()
street_infront_house = df['street_in_front_of_house'].tolist()
direction = df['direction'].tolist()
law = df['law'].tolist()
address = df['address'].tolist()

area = convert_area(area)
width = convert_area(width)
height = convert_area(height)
street_infront_house = convert_area(street_infront_house)
direction = convert_direction(direction)
price = convert_price(price)
law = convert_law(law)
city, district = convert_address(address)

df['price'] = price
df['area'] = area
df['width'] = width
df['height'] = height
df['street_in_front_of_house'] = street_infront_house
df['direction'] = direction
df['law'] = law
df['city'] = city
df['district'] = district

bedroom_no = df['bedroom_number'].tolist()
floor_no = df['floor_number'].tolist()
for i in range(len(bedroom_no)):
    try:
        bedroom_no[i] = float(bedroom_no[i])
    except:
        bedroom_no[i] = np.nan

for i in range(len(floor_no)):
    try:
        floor_no[i] = float(floor_no[i])
    except:
        floor_no[i] = np.nan
df['bedroom_number'] = bedroom_no
df['floor_number'] = floor_no
df.head()

,price,address,area,floor_number,bedroom_number,is_dinning_room,is_kitchen,is_terrace,is_car_pack,is_owner,type,direction,street_in_front_of_house,width,height,law,city,district
0,57.0,"Phố Trích Sài, Phường Thụy Khuê, Quận Tây Hồ,...",67.0,5.0,3.0,True,True,True,True,False,Nhà mặt tiền,NaN,10.0,NaN,NaN,Sổ hồng/ Sổ đỏ,Hà Nội,Tây Hồ
1,17.5,"Đường Nguyễn Văn Huyên, Phường Nghĩa Đô, Quận...",50.0,7.0,4.0,True,True,True,True,False,Nhà mặt tiền,Nam,8.0,4.0,12.0,Sổ hồng/ Sổ đỏ,Hà Nội,Cầu Giấy
2,16.5,"Đường Nguyễn Khánh Toàn, Phường Quan Hoa, Quậ...",60.0,7.0,20.0,True,True,True,True,False,Nhà trong hẻm,Đông Nam,3.0,6.0,12.0,Sổ hồng/ Sổ đỏ,Hà Nội,Cầu Giấy
3,14.9,"Phố Phùng Chí Kiên, Phường Nghĩa Đô, Quận Cầu...",48.0,6.0,3.0,True,True,True,True,False,Nhà mặt tiền,NaN,6.0,4.0,12.0,Sổ hồng/ Sổ đỏ,Hà Nội,Cầu Giấy
4,21.5,"Phố Trung Kính, Phường Trung Hòa, Quận Cầu Gi...",50.0,7.0,4.0,True,True,True,True,False,Nhà mặt tiền,NaN,10.0,4.1,12.0,Sổ hồng/ Sổ đỏ,Hà Nội,Cầu Giấy


In [7]:
df = df.drop(['address','city','law'], axis =  1)
df = df[df.price.notna()]
df = df.drop('is_owner', axis = 1)
df.drop(df[df.price > 50].index, inplace = True)
df.drop(df[df.area > 400].index, inplace = True)
df.drop(df[df.area <= 0].index, inplace = True)

df.drop(df[df.floor_number > 10].index, inplace = True)
df.drop(df[df.floor_number <= 0].index, inplace = True)

df.drop(df[df.bedroom_number > 10].index, inplace = True)
df.drop(df[df.bedroom_number <= 0].index, inplace = True)

df.drop(df[df.street_in_front_of_house > 25].index, inplace = True)

df.drop(df[df.width > 20].index, inplace = True)
df.drop(df[df.width <= 0].index, inplace = True)

df.drop(df[df.height > 25].index, inplace = True)
df.drop(df[df.height <= 0].index, inplace = True)

features_numerical = df.select_dtypes(exclude=['object', 'bool']).copy()
features_categorical = df.select_dtypes(include=['object', 'bool']).copy()

df = df.drop(['height'], axis=1)

In [8]:
def process_cc(row):
    if row['type'] == 'Căn hộ chung cư' and pd.isna(row['floor_number']):
        row['floor_number'] = 1
    return row
df = df.apply(process_cc, axis=1)

In [9]:
df.select_dtypes(exclude=['object', 'bool']).isna().mean().sort_values(ascending = False)

price                       0.0
area                        0.0
floor_number                0.0
bedroom_number              0.0
street_in_front_of_house    0.0
width                       0.0
dtype: float64

In [10]:
columns = ['width', 'floor_number', 'bedroom_number', 'street_in_front_of_house', 'price', 'area']
for i in columns:
    imp = impute.SimpleImputer()
    df[i] = imp.fit_transform(np.array(df[i]).reshape(-1,1))
    if i == 'floor_number' or i == 'bedroom_number':
        df[i] = df[i].round()

In [11]:
df.fillna('None', inplace = True)
df = df.drop('direction', axis = 1)

def process_type(row):
    if row['type'] not in ['Căn hộ chung cư', 'Nhà trong hẻm']:
        row['type'] = 'Nhà mặt phố'
    return row
df = df.apply(process_type, axis=1)

def process_district(row):
    if row['district'] not in ['Long Biên', 'Hoàng Mai', 'Nam Từ Liêm', 'Bắc Từ Liêm', 'Đống Đa', 'Hai Bà Trưng', 'Thanh Xuân', 'Cầu Giấy', 'Tây Hồ','Đông Anh','Gia Lâm','Hà Đông','Hoàn Kiếm','Ba Đình']:
        row['district'] = 'Ngoại thành'
    return row
df = df.apply(process_district, axis=1)



In [12]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
df['is_dinning_room'] = label_encoder.fit_transform(df['is_dinning_room'])
df['is_kitchen'] = label_encoder.fit_transform(df['is_kitchen'])
df['is_terrace'] = label_encoder.fit_transform(df['is_terrace'])
df['is_car_pack'] = label_encoder.fit_transform(df['is_car_pack'])
#  df['type'] = label_encoder.fit_transform(df['type'])
#  df['district'] = label_encoder.fit_transform(df['district'])



In [14]:
df.to_csv('../data/demo.csv', index=False)

In [15]:
df_demo = df.copy()

# Preprocessing

In [17]:
df_train = pd.read_csv('../data/train.csv')
df_val = pd.read_csv('../data/val.csv')
df_test = pd.read_csv('../data/test.csv')
df = pd.concat([df_train, df_val], axis=0)
df

,price,area,floor_number,bedroom_number,is_dinning_room,is_kitchen,is_terrace,is_car_pack,type,street_in_front_of_house,width,district
0,23.90,80.0,6.0,6.0,1,1,1,1,Nhà mặt phố,8.0,5.000000,Long Biên
1,7.40,59.0,5.0,4.0,0,0,0,0,Nhà mặt phố,3.0,5.180581,Bắc Từ Liêm
2,8.40,45.0,4.0,4.0,0,0,0,0,Nhà trong hẻm,3.0,5.180581,Thanh Xuân
3,5.25,35.0,4.0,3.0,1,1,1,0,Nhà trong hẻm,2.5,3.300000,Hà Đông
4,26.00,55.0,9.0,7.0,1,1,1,1,Nhà trong hẻm,7.0,5.180581,Thanh Xuân
...,...,...,...,...,...,...,...,...,...,...,...,...
644,27.00,125.0,5.0,3.0,1,1,1,1,Nhà mặt phố,8.0,6.000000,Ngoại thành
645,31.00,75.0,8.0,10.0,1,1,1,1,Nhà trong hẻm,6.0,6.200000,Thanh Xuân
646,2.80,85.0,5.0,4.0,0,0,0,0,Nhà mặt phố,4.0,4.000000,Ngoại thành
647,12.00,54.0,4.0,6.0,0,0,0,0,Nhà trong hẻm,3.0,4.300000,Hai Bà Trưng


In [18]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

features_numerical = df.select_dtypes(exclude=['object', 'bool']).copy()
numerical_cols = features_numerical.columns.tolist()

numerical_cols.remove('price')

features_categorical = df.select_dtypes(include=['object', 'bool']).copy()
categorical_cols = features_categorical.columns.tolist()

categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
numerical_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols),
        ('num', numerical_transformer, numerical_cols)     
    ],
    remainder='passthrough' 
)



In [19]:
y_train = df_train['price'].copy()
y_val = df_val['price'].copy()
y_demo = df_demo['price'].copy()

df_train = df_train.drop(['price'], axis = 1)
df_val = df_val.drop(['price'], axis = 1)
df_demo = df_demo.drop(['price'], axis = 1)

y_train.to_numpy()
y_val.to_numpy()
y_demo.to_numpy()

X_train = preprocessor.fit_transform(df_train)
X_val = preprocessor.transform(df_val)
X_test = preprocessor.transform(df_test)
X_demo = preprocessor.transform(df_demo)

In [20]:
X_train.shape, X_val.shape, X_test.shape, X_demo.shape, y_train.shape, y_val.shape, y_demo.shape

((1945, 27), (649, 27), (649, 27), (3, 27), (1945,), (649,), (3,))

# Linear Regression

In [28]:
from sklearn.metrics import r2_score

In [29]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
ridge_model = Ridge(alpha=0.001)
ridge_model.fit(X_train, y_train)
y_demo_pred = ridge_model.predict(X_demo)
y_demo_pred, r2_score(y_demo, y_demo_pred)

(array([19.10245715, 16.26240225, 20.6748824 ]), 0.7690819723194622)

# 2-degree Polynomial Regression

In [31]:
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2)
X_train_poly = poly_features.fit_transform(X_train)
ridge_poly = Ridge(alpha = 10)
ridge_poly.fit(X_train_poly, y_train)
X_demo_poly = poly_features.fit_transform(X_demo)
y_demo_pred = ridge_poly.predict(X_demo_poly)
y_demo_pred, r2_score(y_demo, y_demo_pred)

(array([16.91854678, 12.8100945 , 18.90653021]), 0.4828764277482528)